In [93]:
import re
from nltk.stem import PorterStemmer
porter = PorterStemmer()
# import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
regex = re.compile('[^a-z ]')
regexN = re.compile('[^a-z0-9 ]')


In [94]:
def sanitizeBasic(sent):
    #remove all punctuations and numbers
    sent=regexN.sub(' ', sent)
    sent=re.sub(r"[ ]+", " ", sent)
    return sent

In [95]:
def sanitizeStem(sent):
    #remove all punctuations, numbers and stem as well
    sent=regexN.sub(' ', sent)
    sent=re.sub(r"[ ]+", " ", sent)
    sent= porter.stem(sent)
    return sent

In [96]:
def sanitizeStemStopWords(sent):
    #remove all punctuations, numbers and stop words. Do stemming 
    sent=regexN.sub(' ', sent)
    sent=re.sub(r"[ ]+", " ", sent)
    sent=sent.split(" ")
    sent=' '.join([w for w in sent if w not in stop_words])
    sent= porter.stem(sent)
    
#     sent = re.sub(r"[\*\"“”\n\+\-\/\=\(\)‘•:\[\]\|’\!;\.\,\&\?]+", " ", sent)
#     sent = re.sub(r"[0-9]+", "", sent)
#     sent = re.sub(r"[\']+", "", sent)
#     sent=re.sub(r"[ ]+", " ", sent)
    
    return sent

In [97]:
sanitize=sanitizeStemStopWords

In [98]:
import pandas as pd

In [99]:
data=pd.read_json('Music_reviews_json/reviews_Digital_Music_5.json/Music_Review_train.json', lines=True)

In [100]:
#contains the text data
dat=data[['reviewText']].copy()
# print(dat)

In [101]:
#contains the class data 
df_overall=data[['overall']].astype(int).copy()
# print(df_overall)

In [102]:
#find the frequencies of words in a sentence and returns a dictionary
def findFreq(sent):
    val=sent.split(" ")
    freq=set()
    for s in val:
        if s not in freq:
           freq.add(s)
    return freq

In [103]:
dat['reviewText']=dat['reviewText'].str.lower()
# print(dat)

In [104]:
dat['reviewText']=dat['reviewText'].apply(sanitize)
# print(dat)

In [105]:
freq_df=pd.DataFrame()
freq_df['reviewText']=dat['reviewText'].apply(findFreq)
print(freq_df)

                                              reviewText
0      {, 80s, makes, 1011, voice, bones, fit, praise...
1      {label, , smooth, usual, dedicated, bands, bit...
2      {, cd, seem, quot, nobody, feel, ago, produced...
3      {, alien, emotion, cold, explodes, haunting, s...
4      {, alien, chills, flinch, cold, esque, hauntin...
...                                                  ...
49995  {, listening, reveal, left, cd, great, fix, se...
49996  {, overall, definitely, feel, listen, dark, li...
49997  {, excellent, album, big, song, indication, sa...
49998  {, love, hope, singer, ahead, singing, song, w...
49999  {, establishment, known, makes, unique, bangle...

[50000 rows x 1 columns]


In [106]:
vocab=set()
vocabO={i:set() for i in range(1,numclasses+1)}
num_examples=df_overall.shape[0]
for d in range(num_examples):
    x=freq_df['reviewText'].iloc[d]
    o=df_overall['overall'].iloc[d]
    for v in x:
        if v not in vocab:
            vocab.add(v)
        if v not in vocabO[o]:
            vocabO[o].add(v)


In [127]:
if '' in vocab:
    vocab.remove('')
for i in range(1,numclasses+1):
    if '' in vocabO[i]:
        vocabO[i].remove('')

In [107]:
wordFreqWRTOverall={}
numclasses=5
dict_o={i:0 for i in range(1,numclasses+1)}
for v in vocab:
    wordFreqWRTOverall[v]={i:0 for i in range(1,numclasses+1)}
for d in range(num_examples):
    o=df_overall.iloc[d]['overall']
    s=freq_df.iloc[d]['reviewText']
    for v in s:
#         print(v)
        wordFreqWRTOverall[v][o]=wordFreqWRTOverall[v][o]+1;
#     break;

In [129]:
if '' in wordFreqWRTOverall:
    wordFreqWRTOverall.pop('')

In [122]:
maxV={i:0 for i in range(1,numclasses+1)}
for v in vocab:
    for i in range(1, numclasses+1):
        if(maxV[i]<wordFreqWRTOverall[v][i]):
            maxV[i]=wordFreqWRTOverall[v][i]

In [123]:
print(maxV)

{1: 1293, 2: 1703, 3: 3838, 4: 9266, 5: 16777}


In [124]:
boundVal=0.001
bound={i:boundVal*maxV[i] for i in range(1, numclasses+1)}
print(bound)

{1: 1.293, 2: 1.703, 3: 3.838, 4: 9.266, 5: 16.777}


In [112]:
bound2={i:2 for i in range(1, numclasses+1)}

In [130]:
lessImpWords={i:set() for i in range(1,numclasses+1)}
for i in range(1, numclasses+1):
    for v in vocabO[i]:
        if(bound2[i]>wordFreqWRTOverall[v][i]):
            lessImpWords[i].add(v)

In [131]:
lessImpWords={i:list(lessImpWords[i]) for i in range(1,numclasses+1)}

In [132]:
print(lessImpWords)

{1: ['indefinitely', 'ere', 'chronicsnoop', 'egomaniacs', 'hoodies', 'fathers', 'constitution', 'reptition', 'trey', 'swooping', 'astro', 'lump', 'darkside', 'diskman', 'smother', 'astonishingly', 'satisfaction', 'televised', 'ty', 'liberian', 'reinvigorate', 'remnants', 'olso', 'shudder', 'captioned', 'hobby', 'jolly', 'kanyes', 'village', 'emptied', 'starch', 'israin', 'cretins', 'cindy', 'ckheads', 'brody', 'exhibitionist', 'squashing', 'justifying', 'ipods', 'geritol', 'craziness', '4darkness', 'mini', 'visage', 'nielsen', 'overrates', 'expreiments', 'aaaaaaalllllllll', 'dissenting', 'roux', 'rideing', 'sterio', 'exactle', 'ihop', 'workin', 'terminate', 'hugh', 'distasteful', 'starsdreams', 'breakthrough', 'incorporated', 'wagner', 'proverbial', 'blondes', 'beleieve', 'pteromechanophobic', 'guiness', 'infamously', 'paperdoll', 'uptight', 'vebeen', 'cruisin', 'rejecting', 'jumps', 'sav', 'dross', 'catalysts', 'artista', 'danity', 'hart', 'digest', 'plunges', 'frak', 'reverential', '

In [133]:
import json
with open('lessImpWords2.json', 'w+') as f:
    json.dump(lessImpWords, f)

In [134]:
df=pd.DataFrame.from_dict(wordFreqWRTOverall)

In [136]:
df=df.T

In [137]:
print(df)

                    1  2  3  4  5
dummied             0  1  0  0  0
standoiut           0  0  1  0  0
mitigated           0  0  0  0  1
archetypal          0  1  0  4  5
threefold           0  0  1  0  0
...                .. .. .. .. ..
favsatisfactionhey  0  0  0  0  1
fiorella            0  0  0  0  1
breakaway           0  1  1  5  5
sorrows             0  0  2  1  6
bootied             1  0  0  0  0

[101021 rows x 5 columns]


In [148]:
for i in range(1,numclasses+1):
    print(df.nlargest(10,i))

          1     2     3     4      5
album  1293  1703  3838  9266  16777
like   1240  1561  3375  7641  12388
one     935  1220  2785  7050  13942
music   843   822  1827  4815  10285
good    770  1117  2800  6259   8471
songs   723  1286  3025  6876  11292
song    687  1036  2378  5923  10920
even    682   775  1453  3320   5697
cd      669   721  1590  3830   8517
get     662   820  1740  3884   6978
           1     2     3     4      5
album   1293  1703  3838  9266  16777
like    1240  1561  3375  7641  12388
songs    723  1286  3025  6876  11292
one      935  1220  2785  7050  13942
good     770  1117  2800  6259   8471
song     687  1036  2378  5923  10920
really   519   835  1816  4005   6437
music    843   822  1827  4815  10285
get      662   820  1740  3884   6978
even     682   775  1453  3320   5697
           1     2     3     4      5
album   1293  1703  3838  9266  16777
like    1240  1561  3375  7641  12388
songs    723  1286  3025  6876  11292
good     770  1117  280